In [ ]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


In [ ]:
from IPython.display import Image
import os
root_dir = '/content/drive/My Drive/인공지능/week9'
Image(os.path.join(root_dir, "MNIST.PNG"))

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset)
import torch.optim as optim

import os
import numpy as np
from tqdm import tqdm
from sklearn.metrics import accuracy_score

class MNIST(nn.Module): # 상속 
  def __init__(self, config):
    super(MNIST, self).__init__()

    # 데이터의 width 길이를 저장하는 parameter
    self.width = config["input_width_size"]
    # 데이터의 height 길이
    self.height = config["input_height_size"]

    # 과제에서 사용되는 파라미터
    self.feature_size = config["feature_size"]

    # 분류해야 하는 label 수
    self.num_labels = config["num_labels"]

    # 활성화 함수로 Sigmoid 사용
    self.activation = nn.Sigmoid()

    # Neural Network
    # 입력 feature는 28*28=768의 길이를 가진 벡터(flatten)
    self.layer_1 = nn.Linear(in_features=self.width*self.height, out_features = self.num_labels)  

  def forward(self, input_features, labels=None):
    # input_features: [batch, width*height] # batch개의 데이터를 묶어서 학습
    # labels: [batch] 
    # batch를 사용하면 학습속도가 빨라질 뿐 아니라 일반화 능력 상승 (backpropagation)
    layer_1_output = self.layer_1(input_features)
    # layer_1_output : [batch, num_labels]

    activated_output_1 = self.activation(layer_1_output)
    # activated_output: [batch, num_labels] 

    # 학습 시
    if labels is not None:
      loss_fnc = nn.CrossEntropyLoss() # logit은 2차원, label은 1차원
      logits = activated_output_1 # 실제 모델의 출력값
      loss = loss_fnc(logits, labels)
      return loss
    # 평가 시
    else:
      output = torch.argmax(activated_output_1, -1) # 값이 가장 큰 요소의 index 반환
      # output: [batch]
      return output



<pre>
<h2> 1. MNIST 데이터를 읽고 train_X, train_y // test_X, test_y에 저장 </h2>
</pre>

In [ ]:
Image(os.path.join(root_dir, "mnist_example.PNG"))

In [ ]:
from keras.datasets import mnist
import torch
def load_dataset():
  (train_X, train_y), (test_X, test_y) = mnist.load_data()
  print(train_X.shape) #(60000, 28, 28)
  print(train_y.shape)
  print(test_X.shape) #(10000, 28, 28)
  print(test_y.shape)

  train_X = train_X.reshape(-1, 28*28) # 28*28길이의 행을 가지는 array로 변환 
  print(train_X.shape)
  test_X  = test_X.reshape(-1, 28*28)

  # tensor 객체로 변환
  train_X = torch.tensor(train_X, dtype=torch.float)
  train_y = torch.tensor(train_y, dtype=torch.long)
  test_X = torch.tensor(test_X, dtype=torch.float)
  test_y = torch.tensor(test_y, dtype=torch.long)

  return (train_X, train_y), (test_X, test_y)
  
tmp = load_dataset()

11493376/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)
(60000, 784)


In [ ]:
def tensor2list(input_tensor):
    return input_tensor.cpu().detach().numpy().tolist()

<pre>
<h2> 2. 불러온 데이터를 이용하여 모델 학습 및 평가 </h2>
</pre>

In [ ]:
# test 데이터로 모델을 평가하는 함수
def do_test(model, test_dataloader):
  model.eval()
  predicts, answers = [], []
  for step, batch in enumerate(test_dataloader):
    # .cuda()를 통해 메모리에 업로드
    batch = tuple(t.cuda() for t in batch)

    input_features, labels = batch
    output = model(input_features)

    predicts.extend(tensor2list(output))
    answers.extend(tensor2list(labels))
    
  print("Accuracy : {}".format(accuracy_score(answers, predicts)))

In [ ]:
def train(config):
  # 모델 생성
  model = MNIST(config).cuda()

  # 데이터 Load
  (train_X, train_y), (test_X, test_y) = load_dataset()
  
  # TensorDataset/DataLoader를 통해 batch 단위로 데이터를 나누고 Shuffle
  train_features = TensorDataset(train_X, train_y)
  train_dataloader = DataLoader(train_features, shuffle=True, batch_size=config["batch_size"])

  test_features = TensorDataset(test_X, test_y)
  test_dataloader = DataLoader(test_features, shuffle=True, batch_size=config["batch_size"])

  # 모델을 최적화하기 위한 optimizer 선언
  optimizer = optim.Adam(model.parameters(), lr=0.0001)
  model.train()

  for epoch in range(config["epoch"]):
    # epoch 마다 평균 loss를 출력하기위한 loss 저장 리스트
    losses = []

    for step, batch in enumerate(train_dataloader):
      # batch = (train_X[step], train_y[step])

      # .cuda()를 통해 메모리에 업로드
      batch = tuple(t.cuda() for t in batch)

      # 각 feature 저장
      input_features, labels = batch
      
      # 모델 호출을 통해 loss return
      # forward 함수가 호출되는 것 (nn.Module 상속했기 때문)
      # labels가 들어갔다 = 학습단계
      loss = model(input_features, labels)

      # 역전파 변화도 초기화
      # ==> .backward() 호출 시, 변화도 버퍼에 데이터가 계속 누적
      #     이를 초기화 시켜주는 단계
      optimizer.zero_grad()

      # loss 값으로부터 모델 내부 각 매개변수에 대하여 gradient 계산
      loss.backward()

      # 모델 내부 각 매개변수 가중치 갱신
      optimizer.step()

      # 1000스텝마다 loss 출력
      if (step+1) % 1000 == 0:
        print("{} step processed.. current loss : {}".format(step+1, loss.data.item()))
      # tensor가 아닌 실제 실수값 loss.data.item() append
      losses.append(loss.data.item())
    
    print("Average Loss : {}".format(np.mean(losses)))
    # epoch이 끝날 때 마다, 모델 저장
    torch.save(model.state_dict(), os.path.join(config["output_dir_path"], "epoch_{}.pt".format(epoch + 1)))

    # 지금까지 학습한 가중치로 평가 진행
    do_test(model, test_dataloader)
    

def test(config):
  model = MNIST(config).cuda()

  # 저장된 모델 가중치 Load
  model.load_state_dict(torch.load(os.path.join(config["output_dir_path"], config["trained_model_name"])))

  # 데이터 load
  (_, _), (test_X, test_y) = load_dataset()
  
  test_features = TensorDataset(test_X, test_y)
  test_dataloader = DataLoader(test_features, shuffle=True, batch_size=config["batch_size"])
  
  do_test(model, test_dataloader)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
if(__name__=="__main__"):
    output_dir = os.path.join(root_dir, "output")
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    config = {"mode": "train",
              "trained_model_name":"epoch_{}.pt".format(10),
              "output_dir_path":output_dir,
              "input_width_size":28,
              "input_height_size":28,
              "feature_size": 512,
              "num_labels": 10,
              "batch_size":32,
              "epoch":10,
              }

    if(config["mode"] == "test"):
        train(config)
    else:
        test(config)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)
(60000, 784)
Accuracy : 0.8231
